We'll be using the 'requests' library for fetching data from the API and 'psycopg2' to connect with our PostgreSQL database. Then need to establish a connection to our PostgreSQL database. We need to define crucial connection details such as the host, database name, username, password, and port number. I  initiate a connection and create a cursor, our tool to execute SQL commands within the database. It is like a virtual pointer moving through our data.

The next step involves setting up our database. We'll create a table named 'Energy' with specific columns like 'timestamp,' 'reportingGroup,' 'locationName,' 'value,' and 'unit.' These columns define the structure of our data. If the 'Energy' table already exists, we drop it and create a new one.

Our table is ready, and it's time to start populating it. We define the values we want to insert – a timestamp, reporting group, location name, value, and unit. This set of information could represent energy consumption data for a specific location at a given time.

I used python Exception handling method to handle potential errors that may occur during the database interaction. It is a good programming practice.  If everything goes smoothly, we close the cursor and the connection.
To fetching data from an external sourcewe, I use the 'requests' library to make an HTTP GET request to a Nuuka Cloud API endpoint. The endpoint contains specific parameters defining the data we want to retrieve, such as the location name, reporting group, and timeframe.

To check if our request was successful, we print the HTTP status code – a code of 200 indicates success in HTTP terms. Additionally, we print the JSON response from the API. JSON provides a structured way of presenting the data we requested.

In [1]:
# Install the 'psycopg2' library for connecting to PostgreSQL
! pip install psycopg2
! pip install requests

In [13]:
import requests
import psycopg2
from datetime import datetime

In [12]:
def convert_to_timestamp(date_str):
    return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S')

In [4]:
def create_insert_script(timestamp, reportingGroup, locationName, value, unit, cur):
    modified_timestamp = convert_to_timestamp(timestamp)


    insert_data = {
        "timestamp": modified_timestamp,
        "reportingGroup": reportingGroup,
        "locationName": locationName,
        "value": value,
        "unit": unit
    }
    
    insert_query = """
    INSERT INTO Electricity ("timestamp", "reportingGroup", "locationName", value, unit)
    VALUES (%s, %s, %s, %s, %s)
    """
    cur.execute(insert_query, (
        insert_data["timestamp"],
        insert_data["reportingGroup"],
        insert_data["locationName"],
        insert_data["value"],
        insert_data["unit"]
    ))

In [5]:
# Create a connection to the PostgreSQL database
conn = psycopg2.connect(dbname="postgres", user="postgres", password="2352", host="localhost")

# Define database connection details
hostname = "localhost"
database = "postgres"
username = "postgres"
pwd = "2352"
port_id = 5432

# Initialize connection and cursor variables
conn = None
cur = None

# Try to establish a connection with exception handling
try:
    conn = psycopg2.connect (
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id
    )
    
    # Create a cursor for database operations
    cur = conn.cursor()

    # Drop the 'Energy' table if it exists
    cur.execute("DROP TABLE IF EXISTS Electricity")

    # Create the "electricity" table with specific columns and data types
    electricity_script = """CREATE TABLE IF NOT EXISTS electricity (
    
                    "timestamp" timestamp without time zone,
                    "reportingGroup" "char",
                    "locationName" character varying COLLATE pg_catalog."default",
                    value real,
                    unit "char"
            )"""
    cur.execute(electricity_script)
    conn.commit()

    property_script = """ CREATE TABLE IF NOT EXISTS public.property
                        (
                            "locationName" character varying COLLATE pg_catalog."default",
                            "property_Code" character varying COLLATE pg_catalog."default",
                            "propertyName" character varying COLLATE pg_catalog."default"
                        )"""

    cur.execute(property_script)
    conn.commit()
    
    
    prop_elect_script = """CREATE TABLE IF NOT EXISTS property_electricity 
                        (
    
                    "property_Code" character varying COLLATE pg_catalog."default",
                    "propertyName" character varying COLLATE pg_catalog."default",
                    "locationName" character varying COLLATE pg_catalog."default",
                    "timestamp" timestamp without time zone,
                    "reportingGroup" "char",
                    "locationName" character varying COLLATE pg_catalog."default",
                    value real,
                    unit "char"
                )"""
    cur.execute(prop_elect_script)
    conn.commit()
    
    
    
except Exception as error: 
    # Print any exception that occurs during the database operations
    print(error)

# finally:
#     # Close the cursor and connection if they are open
#     if cur is not None:
#         cur.close()
#     if conn is not None:
#         conn.close()



In [6]:
PROPERTY_NAME = [
    "1512 Malmin raitti 3",
    "2194 Lp Isonneva",
    "3504 Helsingin Tennispalatsi Koy",
    "4081 Lpk Impivaara",
    "4541 Finlandia-talo"
]
START_DAY = "2020-01-15"
END_DAY = "2020-01-28"


In [7]:
def fetch_info(property_name):
    
    
    # Make an HTTP GET request to a Nuuka Cloud API endpoint
    response = requests.get(f"https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString={property_name}&ReportingGroup=Electricity&StartTime={START_DAY}&EndTime={END_DAY}")
    # Print the HTTP status code for testing purposes
    print(response.status_code)

    # Print the JSON response from the API (assuming it's a successful response)
    print(response.json())  # 200 is standard HTTP successful GET response
    data = response.json()

    for entity in data:
        timestamp  = entity["timestamp"] 
        reportingGroup  =entity["reportingGroup"] 
        locationName = entity["locationName"] 
        value  = entity["value"] 
        unit = entity["unit"]
        create_insert_script(timestamp, reportingGroup, locationName, value, unit, cur)
    conn.commit()



# delete all data
cur.execute("DELETE FROM Electricity")

for property_name in PROPERTY_NAME:
    fetch_info(property_name)

200
[{'timestamp': '2020-01-15T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2020-01-16T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2020-01-17T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2020-01-18T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2020-01-19T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2020-01-20T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2020-01-21T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1512 Malmin raitti 3', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '

200
[{'timestamp': '2020-01-15T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '4541 Finlandia-talo', 'value': 4838.1799999999985, 'unit': 'kWh'}, {'timestamp': '2020-01-16T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '4541 Finlandia-talo', 'value': 6567.580000000001, 'unit': 'kWh'}, {'timestamp': '2020-01-17T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '4541 Finlandia-talo', 'value': 6372.690000000001, 'unit': 'kWh'}, {'timestamp': '2020-01-18T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '4541 Finlandia-talo', 'value': 4047.0299999999993, 'unit': 'kWh'}, {'timestamp': '2020-01-19T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '4541 Finlandia-talo', 'value': 2794.7400000000002, 'unit': 'kWh'}, {'timestamp': '2020-01-20T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '4541 Finlandia-talo', 'value': 6900.459999999998, 'unit': 'kWh'}, {'timestamp': '2020-01-21T00:00:00', 'reportingGroup': 'Electricity', 'loc

list_api = "https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List"

energy_api = "https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1000%20Hakaniemen%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31"

def api_data(url):
    response = requests.get(url)
    data = response.json()
    return data

api_data(list_api)
api_data(energy_api)


create_script_3= """CREATE TABLE Property_electricity AS 
                    SELECT * 
                    FROM list_api l
                    INNER JOIN api_data a ON l.locationName = a.locationName
                    WHERE DATE(a.timestamp) BETWEEN '2020-01-15' AND '2020-01-28'"""
cur.execute(create_script_3)
conn.commit()

create_script_3= """CREATE TABLE merge_table AS 
                    SELECT * 
                    FROM list_api l
                    INNER JOIN api_data a ON l.locationName = a.locationName
                    WHERE DATE(a.timestamp) BETWEEN '2020-01-15' AND '2020-01-28'"""

cur.execute(create_script_3)
conn.commit()

In [24]:
def list_script(locationName, propertyName, propertyCode, cur):
    insert_data= {
        "propertyName": propertyName,
        "locationName": locationName,
        "propertyCode": propertyCode,
        
    }
    
    insert_query = """ 
    INSERT INTO public.property(
	"locationName", "propertyName", "propertyCode")
	VALUES (%s, %s, %s);
    """
    
    cur.execute(insert_query, (
        insert_data["locationName"],
        insert_data["propertyName"],
        insert_data["propertyCode"],
        
    ))
    print(insert_data)
      #Drop the 'property' table if it exists
   
    
     
    
# Create a connection to the PostgreSQL database
conn = psycopg2.connect(dbname="postgres", user="postgres", password="2352", host="localhost")

# Define database connection details
hostname = "localhost"
database = "postgres"
username = "postgres"
pwd = "2352"
port_id = 5432

# Initialize connection and cursor variables
conn = None
cur = None

# Try to establish a connection with exception handling

conn = psycopg2.connect (
    host=hostname,
    dbname=database,
    user=username,
    password=pwd,
    port=port_id
)
    
# Create a cursor for database operations
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS property")
    
property_script = """ CREATE TABLE IF NOT EXISTS public.property
                        (
                            "locationName" character varying COLLATE pg_catalog."default",
                            "propertyCode" character varying COLLATE pg_catalog."default",
                            "propertyName" character varying COLLATE pg_catalog."default"
                        )"""

cur.execute(property_script)
conn.commit()
# Make an HTTP GET request to a Nuuka Cloud API endpoint
property_list= requests.get(f"https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List")
#print(property_list.json())
property_data= property_list.json()
    
for item in property_data: 
    locationName = item["locationName"]
    propertyName = item["propertyName"]
    propertyCode = item["propertyCode"]
    list_script(locationName, propertyName, propertyCode, cur)

conn.commit()
    


{'propertyName': '1000 Hakaniemen kauppahalli', 'locationName': '1000 Hakaniemen kauppahalli', 'propertyCode': '091-011-9902-0101'}
{'propertyName': '1001 Hietalahden kauppahalli', 'locationName': '1001 Hietalahden kauppahalli', 'propertyCode': '091-004-9902-0008'}
{'propertyName': '1002 Vanha kauppahalli', 'locationName': '1002 Vanha kauppahalli', 'propertyCode': '091-003-9906-0101'}
{'propertyName': '1037 Vuotalo', 'locationName': '1037 Vuotalo', 'propertyCode': '091-054-0179-0003'}
{'propertyName': '1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen', 'locationName': '1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen', 'propertyCode': '091-040-0024-0003'}
{'propertyName': '1508 Monitoimitalo Puustelli', 'locationName': '1508 Monitoimitalo Puustelli', 'propertyCode': '091-033-0232-0003, 091-033-0260-0001'}
{'propertyName': '1509 Lpk Karvikka ja rppk Lemmikki', 'locationName': '1509 Lpk Karvikka ja rppk Lemmikki', 'propertyCode': '091-038-0055-0010'}
{'propertyName': '1511 Pa

{'propertyName': '6795 Paviljonki- lpk Violanpuisto', 'locationName': '6795 Paviljonki- lpk Violanpuisto', 'propertyCode': '091-021-9903-0003'}
{'propertyName': '6818 Kannelmäen peruskoulun lisätilat', 'locationName': '6818 Kannelmäen peruskoulun lisätilat', 'propertyCode': '091-033-0114-0001'}
{'propertyName': '6821 Suutarilan ala-asteen lisätila, päiväkoti', 'locationName': '6821 Suutarilan ala-asteen lisätila, päiväkoti', 'propertyCode': '091-040-0024-0003'}
{'propertyName': '6825 Arabian pk lisätila', 'locationName': '6825 Arabian pk lisätila', 'propertyCode': '091-408-0001-0000'}
{'propertyName': '6828 Paviljongit- Konalan koulu ja päiväkoti Karhi', 'locationName': '6828 Paviljongit- Konalan koulu ja päiväkoti Karhi', 'propertyCode': '091-032-0061-0002'}
{'propertyName': '6838 Paviljonki- lpk Vilppula ja Suvi', 'locationName': '6838 Paviljonki- lpk Vilppula ja Suvi', 'propertyCode': '091-038-0078-0004'}
{'propertyName': '6842 Paviljonki- lpk Kallahti lisätila', 'locationName': '68